# Connection à la base de données 'fcd'

In [1]:
import pandas as pd
import pymongo

client = pymongo.MongoClient('localhost')
collection = client.fcd.assets

print('done')

done


# Lecture de la base de donnée

In [18]:
import pprint
cursor = collection.find()
entries = list(cursor)

P = []
for row in entries :
    point = {}
    point['asset_id'] = row['asset_id']
    point['recorded_at'] = row['recorded_at']
    point['latitude'] = row['location']['geo']['coordinates'][1]
    point['longitude'] =row['location']['geo']['coordinates'][0]
    P.append(point)
        
        
P = sorted(P, key=lambda point: point['recorded_at'])   # tri selon timestamp(recorded_at)
print('done')

done


# Application de l'algorithme des stay points

In [19]:

# in minutes
def time_gap(current, new):
    
    td = abs(current - new)
    res = td/60
    return res

# in meters
def distance(lat1, lat2, lon1, lon2):
    R = 6372800  # Earth radius in meters
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2) ** 2
    d = 2 * R * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return d


def algorithm_stayPoint_detection(P, distThreh, timeThreh):

    i = 0
    pointNum = len(P)
    print('taille', pointNum)
    SP = []

    while i < pointNum-1:
        
        j = i + 1
        
        while j < pointNum:
            dist = distance(float(P[i]['latitude']), float(P[j]['latitude']),
                            float(P[i]['longitude']), float(P[j]['longitude']))

            if dist > distThreh:
                break
                
            j+=1
        
        j-=1
        
        deltaT = time_gap(P[i]['recorded_at'], P[j]['recorded_at'])
        if deltaT > timeThreh:
            S = {}
            S['arv'] = i
            S['lev'] = j
            SP.append(S)
            i=j
        else:
            i+=1

            
    return SP


S = algorithm_stayPoint_detection(P, 200, 30)
print('done')

taille 118653
done


# Séparation des différents trips

In [20]:
border = 0
trips = []

for stayP in S:
    
    if stayP['arv'] <= border:
        border = stayP['lev']
    else:
        S ={}
        S['arv'] = border
        S['lev'] = stayP['arv']
        trips.append(S)
        border = stayP['lev']
        

if border < len(P)-1:
    S ={}
    S['arv'] = border
    S['lev'] = len(P)-1
    trips.append(S)

print('done')

done


# Création d'un dictionnaire des points sans Stay point

In [21]:
data=[]
num_trip=0
for trip in trips:
    num_trip += 1
    for i in range(trip['arv'], trip['lev']+1):
        point = {}
        point['trip']=num_trip
        point['asset_id'] = P[i]['asset_id']
        point['recorded_at'] = P[i]['recorded_at']        
        g = {}
        coord = []
        coord.append(float(P[i]['longitude']))
        coord.append(float(P[i]['latitude']))
        g['coordinates'] = coord
        g['type'] = "Point"
        loc = {}
        loc['geo'] = g
        point['location'] = loc
        data.append(point)

    
print('done')

done


# Index géo spatial 

In [22]:
import pymongo
from pymongo import GEOSPHERE

client = pymongo.MongoClient('localhost')
collection = client.fcd.trips
resp = collection.create_index([("location.geo", GEOSPHERE)])
print ("index response:", resp)

index response: location.geo_2dsphere


# Insertion dans la base de donnée 

In [23]:

client = pymongo.MongoClient('localhost',27017)
mydb = client["fcd"]
collection = mydb["trips"]

print('insert')
collection.insert_many(data)
print('done')

insert
done


# Lecture dans la base

In [28]:
import pandas as pd
cursor = collection.find()
entries = list(cursor)

#entries[:]

df = pd.DataFrame(entries)
df.head()
print('\ndone')


done
